# Bixi Telemetry Data Warehouse
### Data Engineering Capstone Project

#### Project Summary
The goal of this project is the build a data warehouse for Bixi, a Montreal-based bike-sharing service. The modeled data will then be used to analyze ride pattern and availability of bikes at every station, enabling Bixi to optimize replenishment activities.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
import pandas as pd
import boto3
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import explode

In [ ]:
GBFS_SOURCE = "s3a://bixi-gbfs-data"
GBFS_DESTINATION = "s3a://bixi-gbfs-data-parquet"

STATION_INFO_DATA = "https://gbfs.velobixi.com/gbfs/en/station_information.json"
TRIP_APRIL_DATA = "https://sitewebbixi.s3.amazonaws.com/uploads/docs/biximontreal-rentals-2021-04-87aaed.zip"

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
The main dataset used will be a json output of the Bixi API, which exposes their [GBFS (General Bikeshare Feed Specification)](https://github.com/NABSA/gbfs) feed. The API was queries every 5 minutes for 3 weeks, using a Python script scheduled by a cron job. The response json was then uploaded to an Amazon S3 bucket.

We start by provisioning the AWS access ID and secret key in the spark config and instantiate a spark session

In [ ]:
config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))
access_id = config.get("default", "aws_access_key_id") 
access_key = config.get("default", "aws_secret_access_key")

In [ ]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
conf.set("fs.s3a.access.key", access_id)
conf.set("fs.s3a.secret.key", access_key)

spark = SparkSession.builder.config(conf=conf).getOrCreate()

Once the Spark session is created, all the `json` files in the S3 bucket are then read into a Spark dataframe. The nested station data is exploded to individual rows and split by column. The unix epoch timestamp in column `last_updated` is converted to a timestamp, then several time parts are expanded into multiple column, which will later be used to partition the parquet output 

In [ ]:
df = spark.read.json(f"{GBFS_SOURCE}/*.json")

In [ ]:
df_flat = (df
    .select("last_updated",explode("data.stations").alias('stations'))
    .select("last_updated", "stations.*")    
)

In [ ]:
df_flat = df_flat \
    .withColumn("last_updated_dt", F.from_unixtime("last_updated")) \
    .withColumn("year", F.year("last_updated_dt")) \
    .withColumn("month", F.month("last_updated_dt")) \
    .withColumn("day", F.dayofmonth("last_updated_dt")) \
    .withColumn("day_of_week", F.dayofweek("last_updated_dt")) \
    .withColumn("hour", F.hour("last_updated_dt"))

Lastly, the dataframe is written back to S3 as a `parquet` file, partitioned by year, month, day

In [ ]:
df_flat.write \
    .partitionBy('year','month','day') \
    .parquet(path = f"{GBFS_DESTINATION}/gbfs.parquet", mode = "overwrite") 

Next, the Bixi API is called again to obtain the latest station metadata

In [ ]:
json_output = "data/station_info.json"
with open(json_output, 'w') as f:
    json.dump(requests.get(STATION_INFO_DATA).json(),f)

In [ ]:
df_station = spark.read.json(json_output)

In [ ]:
df_station_flat = df_station \
    .select(explode("data.stations").alias('stations')) \
    .select("stations.lat"
        , "stations.lon"
        , 'stations.station_id'
        , "stations.name"
    )

The dataframe is written back to S3 as a parquet file called `stations.parquet`

In [ ]:
df_station_flat.write.parquet(
    path = f"{GBFS_DESTINATION}/stations.parquet", 
    mode = "overwrite"
)

The last dataset is trip data from the month april 2021, which is the same timeframe as the GBFS feed, which needs to be extracted from a `zip` file, read into a spark dataframe from a `csv` file, enriched with time based columns, and finally written back to S3 as a `parquet` file, partition by year, month, day, similar to the GBFS feed.

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.